In [ ]:
import geemap
import ee

Map = geemap.Map()
Map

In [ ]:
import os
os.chdir(r'H:\Monarch\Work\Gansu_PV_2020_map')

In [ ]:
shp_file = r'gansu_pv_roi.shp'
shp = geemap.shp_to_ee(shp_file)

In [ ]:
Map.addLayer(shp, {}, 'shp')
Map.center_object(shp, 6)
roi = shp.geometry()

In [ ]:
image = ee.Image("projects/ee-2431566134liumonarch/assets/Gansu_pv_2020_part3_map")
Map.addLayer(image, {'palette':"blue"},'pv_final')

In [ ]:
img_c = image.connectedPixelCount(700, False)
# Map.addLayer(img_c.gte(150), {'min':1, 'max': 1, 'palette': ['ff0000']},'img_c')
# geemap.ee_export_image(img_c.clip(window), 'img_c.tif', 30, region=window)

In [ ]:
image_mask = image.mask(img_c.gte(700))
# Map.addLayer(image_mask, {'palette':"red"},'pv_final_mask')

In [ ]:
image_mask_vector = image_mask.reduceToVectors(**{
    'geometry':roi,
    'scale': 10,
    'maxPixels': 1e13
})

In [ ]:
Map.addLayer(image_mask_vector, {'color':'green'}, 'vector')

In [ ]:
def feature_area_per(feature):
    feature = ee.Feature(feature)
    area = feature.area(0.01)
    perimeter = feature.perimeter(0.01)
    feature = feature.set('area', area, 'perimeter', perimeter, 'ratio', area.divide(perimeter))
    return feature

In [ ]:
feature_image = image_mask_vector.map(feature_area_per)

In [ ]:
Map.addLayer(feature_image.filter(ee.Filter.gt('ratio', 45)), {'color':'red'}, 'feature_image')

In [ ]:
geemap.ee_to_shp(feature_image.filter(ee.Filter.gt('ratio', 45)), r'feature_image_f.shp')

In [ ]:
geemap.ee_to_shp(feature_image, r'feature_image.shp')